## Imports

In [38]:
import random
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser

## Define your desired data structure.

In [21]:
class Employee(BaseModel):
   emp_id: int = Field(title="Employee ID", description="The unique identifier for the employee")
   emp_name : str = Field(title="Employee Name", description="The name of the employee")
   emp_age : int = Field(title="Employee Age", description="The age of the employee")
   emp_dept : str = Field(title="Employee Department", description="The department of the employee")
   emp_location : str = Field(title="Employee Location", description="The location of the employee")

## Set up a parser

In [24]:
parser = PydanticOutputParser(pydantic_object=Employee)

In [39]:
str_parser = StrOutputParser()

## LLM

In [27]:
from langchain_huggingface import HuggingFaceEndpoint
import os
repo_id = "mistralai/Mistral-Nemo-Instruct-2407" # "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=128,
    temperature=0.5,
    huggingfacehub_api_token=os.environ["API_TOKEN"],
)

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
d:\Learning\Python\Generative AI\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to C:\Users\wotcw4\.cache\huggingface\token
Login successful


## Prompts

In [29]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

## Chain Invoking

In [41]:
chain = prompt | llm | str_parser

ret = chain.invoke({"query": "Aditya Ghumatkar, age 27 is an employee of Siemens DISW.His employee id is 45654, he is working in the Innovation department and is located in Pune."})

## Convert the class object to JSON response 

In [46]:
s = parser.get_format_instructions()

jp = JsonOutputParser()
emp_as_json = jp.invoke(s)

#save wi_as_json to file
import json
with open('data/emp_data.json','w') as f:
    f.write(json.dumps(emp_as_json, indent=4))

In [47]:
j = jp.invoke(str(ret))


In [48]:
j

{'emp_id': 45654,
 'emp_name': 'Aditya Ghumatkar',
 'emp_age': 27,
 'emp_dept': 'Innovation',
 'emp_location': 'Pune'}